# Setting up environment.

In [1]:
!pip install -q uv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.8/17.8 MB 88.3 MB/s eta 0:00:00:00:0100:01


In [2]:
!uv venv gen-ai

Using CPython 3.11.11 interpreter at: /usr/bin/python3
Creating virtual environment at: gen-ai
Activate with: source gen-ai/bin/activate


In [3]:
!source gen-ai/bin/activate

In [4]:
!uv pip install -q langchain langchain-huggingface langchain-community langchain unstructured langgraph langchain-core transformers "torch[cuda]" "unstructured[docs]" 

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading HuggingFace Model using Langchain_HuggingFace

### Qwen3-0.6B mode

In [6]:
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_huggingface import ChatHuggingFace
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

2025-06-15 17:31:33.627341: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750008693.843208      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750008693.909590      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
llm = HuggingFacePipeline.from_model_id(
    model_id="Qwen/Qwen3-0.6B",
    task="text-generation",
    pipeline_kwargs={
        "temperature": 0.6
    }
)

tokenizer_config.json:   0%|          | 0.00/9.73k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
model = ChatHuggingFace(llm=llm)

In [11]:
result = model.invoke(
    "explain big bang theory to me like i'm 5?"
)

In [12]:
result

AIMessage(content='<|im_start|>user\nexplain big bang theory to me like i\'m 5?<|im_end|>\n<|im_start|>assistant\n<think>\nOkay, so the user wants me to explain "Big Bang Theory" to someone like a', additional_kwargs={}, response_metadata={}, id='run--c2e30d3f-adae-49cd-9b7d-c754e47a8312-0')

In [9]:
print(result.content)

<|im_start|>user
What are large language models?<|im_end|>
<|im_start|>assistant
<think>
Okay, the user asked, "What are large language models?" I need to explain this


# Inferencing Qwen model using Huggingface

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "nvidia/Nemotron-Research-Reasoning-Qwen-1.5B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)


tokenizer_config.json:   0%|          | 0.00/9.73k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

...

model-00001-of-00005.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

thinking content: <think>
Okay, the user is asking for a short introduction to large language models. Let me start by recalling what I know about them. Large language models are AI systems trained on vast amounts of text data, right? They can understand and generate human-like text. But I need to make sure I cover the basics without getting too technical.

First, I should mention what they are: AI models trained on massive datasets. Then, their main capabilities—like understanding and generating text. Maybe include examples of tasks they can perform, such as answering questions, writing stories, coding. Also, mention the scale, like billions of parameters. Oh, and the training process involves learning patterns in the data. Should I explain what parameters are? Maybe not in detail since it's a short intro. Also, note that they're used in various applications like chatbots, customer service, content creation. But keep it concise. Let me check if I'm missing anything important. Maybe men

# Mistral_AI using Langchain_HuggingFace

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_huggingface import ChatHuggingFace
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import HumanMessage
import torch

In [8]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
api_key = ""

# loading tokenizer and model huggingface way
tokenizer = AutoTokenizer.from_pretrained(model_id, token=api_key)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto", token=api_key)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1000)
llm = HuggingFacePipeline(pipeline=pipe)
chat_model = ChatHuggingFace(llm=llm)

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


In [10]:
from pydantic import BaseModel, Field
from typing import Optional

In [16]:
# Output class structure
class ModelOutput(BaseModel):
    answer: str

def parse_mistral_output(raw_output: str) -> ModelOutput:
    special_token = "[/INST]"

    try:
        # index where special token is found
        start_index = raw_output.rindex(special_token) + len(special_token)

        extracted_text = raw_output[start_index:]

        clean_extracted_text = extracted_text.strip()

        return ModelOutput(answer=clean_extracted_text)

    except ValueError:
        print(f"Warning: {special_token} not found in text!")
        return ModelOutput(answer=raw_output.strip())

In [17]:
prompt_template = ChatPromptTemplate([
    ("system", "You are a Theoretical Physicist with expertise in string theory"),
    ("human", "{question}")
])

model_output_parser = StrOutputParser()
mistral_output_parser = RunnableLambda(parse_mistral_output)
chat_chain = prompt_template | chat_model | model_output_parser | mistral_output_parser

In [18]:
question = "What is String Theory? Give me a detailed example."
response = chat_chain.invoke({"question": question})

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [19]:
print(response)

answer="String Theory is a theoretical framework in which the point-like particles of particle physics are replaced by one-dimensional objects called strings. It is a candidate for a unified theory of all fundamental forces and forms of matter, including gravity, electromagnetism, and the nuclear forces.\n\nIn String Theory, strings can vibrate at different frequencies, and these vibrations correspond to different fundamental particles. For example, a string vibrating at a certain frequency could correspond to a photon (the particle of light), while a string vibrating at a different frequency could correspond to an electron.\n\nLet's consider a simple example to illustrate this idea. Imagine a guitar string. When you pluck the string, it vibrates at a certain frequency, producing a note. If you pluck the string harder or shorter, the frequency changes, producing a different note. In the same way, a string in String Theory can vibrate at different frequencies, each frequency correspondi

In [20]:
print(response.answer)

String Theory is a theoretical framework in which the point-like particles of particle physics are replaced by one-dimensional objects called strings. It is a candidate for a unified theory of all fundamental forces and forms of matter, including gravity, electromagnetism, and the nuclear forces.

In String Theory, strings can vibrate at different frequencies, and these vibrations correspond to different fundamental particles. For example, a string vibrating at a certain frequency could correspond to a photon (the particle of light), while a string vibrating at a different frequency could correspond to an electron.

Let's consider a simple example to illustrate this idea. Imagine a guitar string. When you pluck the string, it vibrates at a certain frequency, producing a note. If you pluck the string harder or shorter, the frequency changes, producing a different note. In the same way, a string in String Theory can vibrate at different frequencies, each frequency corresponding to a diff

In [21]:
prompt_template2 = ChatPromptTemplate([
    ("system", "You are a Senior Data Scientist. You have a working experience of 15+ years."),
    ("user", "{question}"),
])

chat_chain2 = prompt_template2 | chat_model | model_output_parser | mistral_output_parser

In [22]:
question2 = "list all data cleaning and handling techniques?"
response2 = chat_chain2.invoke({"question": question2})

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [23]:
print(response2.answer)

As a Senior Data Scientist with over 15 years of experience, I have employed various data cleaning and handling techniques throughout my career. Here is a list of some common techniques:

1. **Handling Missing Values**:
   - Removing rows or columns with missing values (depending on the data and the analysis)
   - Filling missing values with mean, median, mode, or other statistical measures
   - Using advanced techniques like multiple imputation, regression imputation, or k-nearest neighbors imputation

2. **Data Normalization and Standardization**:
   - Min-Max Scaling: Scaling data between 0 and 1
   - Z-Score Standardization: Scaling data to have a mean of 0 and standard deviation of 1
   - Log Transformation: Transforming data to reduce skewness and improve model performance

3. **Outlier Detection and Handling**:
   - Identifying outliers using statistical methods (e.g., Z-score, IQR) or machine learning algorithms
   - Removing outliers or replacing them with mean, median, or oth

### Parsing Mistral Output

In [16]:
import pydantic
from typing import Optional

# 1. Define the Pydantic Model for the parsed output
# This class defines the structure of our final, clean output.
# We want to extract the model's answer into a single string field.
class ModelOutput(pydantic.BaseModel):
    """Pydantic model to hold the parsed answer from the LLM."""
    answer: str
    
    # You could add other fields here if you wanted to extract more structured data.
    # For example:
    # techniques: Optional[list[str]] = None

# 2. Create a parsing function
def parse_mistral_output(raw_output: str) -> ModelOutput:
    """
    Parses the raw string output from the Mistral model to extract the final answer.

    Args:
        raw_output: The complete string returned by the language model.

    Returns:
        A Pydantic ModelOutput object containing the clean answer.
    """
    # The model's actual answer comes after the [/INST] tag.
    # We find the position of this tag and take the substring that follows it.
    inst_marker = "[/INST]"
    
    try:
        # Find the index where the marker ends
        answer_start_index = raw_output.rindex(inst_marker) + len(inst_marker)
        
        # Extract the substring after the marker
        extracted_text = raw_output[answer_start_index:]
        
        # Clean up any leading/trailing whitespace
        clean_text = extracted_text.strip()
        
        # Create and return the Pydantic model instance
        return ModelOutput(answer=clean_text)

    except ValueError:
        # This will happen if the "[/INST]" marker is not in the output.
        # In this case, we can assume the whole output is the answer,
        # or handle it as an error. For robustness, we'll treat it as the answer.
        print("Warning: '[/INST]' marker not found. Treating entire output as the answer.")
        return ModelOutput(answer=raw_output.strip())


# --- Example Usage ---

# This is the raw output you provided from the Hugging Face model
sample_output = """
<s>[INST] You are a Senior Data Scientist. You have a working experience of 15+ years.

list all data cleaning and handling techniques?[/INST] As a Senior Data Scientist with over 15 years of experience, I have employed various data cleaning and handling techniques throughout my career. Here is a list of some common techniques:

1. **Handling Missing Values**:
   - Removing rows or columns with missing values (depending on the data and the analysis)
   - Filling missing values with mean, median, mode, or other statistical measures
   - Using advanced techniques like multiple imputation, regression imputation, or k-nearest neighbors imputation

2. **Data Normalization and Standardization**:
   - Min-Max Scaling: Scaling data between 0 and 1
   - Z-Score Standardization: Scaling data to have a mean of 0 and standard deviation of 1
   - Log Transformation: Transforming data to reduce skewness and improve model performance

3. **Outlier Detection and Handling**:
   - Identifying outliers using statistical methods (e.g., Z-score, IQR) or machine learning algorithms
   - Removing outliers or replacing them with mean, median, or other statistical measures
   - Using robust statistical methods that are less sensitive to outliers
"""

# Parse the output
parsed_data = parse_mistral_output(sample_output)

# Now you have a structured object, and you can access the clean answer easily
print("--- Parsed LLM Answer ---")
print(parsed_data.answer)

# You can also easily convert the Pydantic object to a dictionary if needed
print("\n--- As Dictionary ---")
print(parsed_data.model_dump())

--- Parsed LLM Answer ---
As a Senior Data Scientist with over 15 years of experience, I have employed various data cleaning and handling techniques throughout my career. Here is a list of some common techniques:

1. **Handling Missing Values**:
   - Removing rows or columns with missing values (depending on the data and the analysis)
   - Filling missing values with mean, median, mode, or other statistical measures
   - Using advanced techniques like multiple imputation, regression imputation, or k-nearest neighbors imputation

2. **Data Normalization and Standardization**:
   - Min-Max Scaling: Scaling data between 0 and 1
   - Z-Score Standardization: Scaling data to have a mean of 0 and standard deviation of 1
   - Log Transformation: Transforming data to reduce skewness and improve model performance

3. **Outlier Detection and Handling**:
   - Identifying outliers using statistical methods (e.g., Z-score, IQR) or machine learning algorithms
   - Removing outliers or replacing them